<a href="https://colab.research.google.com/github/praffuln/langchain/blob/master/langchain-2/langchain_8_customQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openAI
!pip install wikipedia
!pip install huggingface_hub
!pip install InstructorEmbedding
!pip install google-search-results
!pip install unstructured
!pip install libmagic
!pip install python-magic
!pip install python-magic-bin
#Install faiss Packages
!pip install faiss-cpu
!pip install sentence-transformers
!pip install wolframalpha

ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


### API KEYS

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

# nltk.download('averaged_perceptron_tagger')

# pip install unstructured
# Other dependencies to install https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/unstructured_file.html
# pip install python-magic-bin
# pip install chromadb

In [ ]:
# Get your loader ready
loader = DirectoryLoader('/content/sample_data/test_directory/', glob='**/*.txt')

In [ ]:
# Load up your text into documents
documents = loader.load()

In [ ]:
# Get your text splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [ ]:
# Split your documents into texts
texts = text_splitter.split_documents(documents)
texts

[Document(page_content="April 2020I recently saw a\n\nvideo\n\nof TV journalists and politicians confidently\n\nsaying that the coronavirus would be no worse than the flu. What\n\nstruck me about it was not just how mistaken they seemed, but how\n\ndaring. How could they feel safe saying such things?The answer, I realized, is that they didn't think they could get\n\ncaught. They didn't realize there was any danger in making false\n\npredictions. These people constantly make false predictions, and", metadata={'source': '/content/sample_data/test_directory/cred.txt'}),
 Document(page_content="get away with it, because the things they make predictions about\n\neither have mushy enough outcomes that they can bluster their way\n\nout of trouble, or happen so far in the future that few remember\n\nwhat they said.An epidemic is different. It falsifies your predictions rapidly and\n\nunequivocally.But epidemics are rare enough that these people clearly\n\ndidn't realize this was even a possibi

In [ ]:
# Turn your texts into embeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# Get your docsearch ready
docsearch = FAISS.from_documents(texts, embeddings)

In [ ]:
# Load up your LLM
llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 0.5}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
# Create your Retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
# Run a query
query = "How to explain all the time he spent on alchemy and theology? "
qa.run(query)

'smart people are often kind of crazy'

### Sources

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=docsearch.as_retriever(),
                                return_source_documents=True)
query = "What did McCarthy discover?"
result = qa({"query": query})

In [ ]:
result['result']

'no one else had noticed'

In [ ]:
result['source_documents']

[Document(page_content="January 2017Because biographies of famous scientists tend to\n\nedit out their mistakes, we underestimate the\n\ndegree of risk they were willing to take.\n\nAnd because anything a famous scientist did that\n\nwasn't a mistake has probably now become the\n\nconventional wisdom, those choices don't\n\nseem risky either.Biographies of Newton, for example, understandably focus\n\nmore on physics than alchemy or theology.\n\nThe impression we get is that his unerring judgment", metadata={'source': '/content/sample_data/test_directory/disc.txt'}),
 Document(page_content='seemed roughly equally promising. No one knew yet\n\nwhat the payoff would be for inventing what we\n\nnow call physics; if they had, more people would\n\nhave been working on it. And alchemy and theology\n\nwere still then in the category Marc Andreessen would\n\ndescribe as "huge, if true. "Newton made three bets. One of them worked. But\n\nthey were all risky.', metadata={'source': '/content/sampl